In this workbook, I have created a deep learning model using Keras API. 

I have used an embedded layer which creates the word2vec vectors for the feature texts. Then added a LSTM layer followed by fully connected layer. The accuracy of the model on new data is 89% 



### Importing the required modules/packages

In [71]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.corpus import stopwords
import string
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
#from mlxtend.feature_selection import ColumnSelector
from sklearn.compose import ColumnTransformer
nltk.download('stopwords')
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading file and looking into the dimensions of data

In [72]:
raw_data = pd.read_csv("SMSSpamData.csv",names=['label','text'], encoding='ISO-8859-1')
pd.set_option('display.max_colwidth',100)


print(f"Shape of Data --> {raw_data.shape}\n")
#print(pd.crosstab(raw_data['label'],columns = 'label'))
#pd.crosstab(raw_data['label'],columns = 'label',normalize=True)
raw_data.head()

Shape of Data --> (5572, 2)



,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


## **Data Processing**

### Functions to Create new features and cleaning the data

In [73]:
## Percentage Count
def punct_pc(text):
    punct_count = sum([1 for char in text if char in string.punctuation])
    return (punct_count/(len(text) - text.count(' ')))*100

## Stem
def clean_data(text):
    punct = "".join([word.lower() for word in text if word not in string.punctuation])
    splt = re.split('\W+',punct)
    txt = [nltk.PorterStemmer().stem(word) for word in splt if word not in nltk.corpus.stopwords.words('english')]
    return txt

    


### Train and Test Split 

In [74]:
## Splitting the Data using Test size 0.2
X_train,X_test,y_train,y_test = train_test_split(raw_data[['text']],raw_data[['label']],test_size=0.2,random_state=23)

In [75]:
y_train

,label
1627,spam
4608,ham
2709,ham
639,ham
4544,ham
...,...
2998,ham
1993,ham
1064,ham
4838,ham


In [76]:
## Covert target to binary
y_train = np.where(y_train == 'spam', 1,0)
y_test = np.where(y_test == 'spam', 1,0)

In [77]:
######################## Create new features Train and Test Data ########################
#- Two new features are created - 
#- 1) text_length (the total length of the text)
#- 2) Punct_pc (the percentage of punctuations in the text)

## Train Data
X_train["punct_pc"] = X_train["text"].apply(lambda x: punct_pc(x))
X_train["text_length"] = X_train["text"].apply(lambda x: len(x)-x.count(' '))

## Test Data
X_test["punct_pc"] = X_test["text"].apply(lambda x: punct_pc(x))
X_test["text_length"] = X_test["text"].apply(lambda x: len(x)-x.count(' '))

### Pipeline to process the Text Data

- Tokenization
- Cleaning
- Normalization
- Lemmatization
- Steaming

Custom Transformer to select the Columns

In [78]:
## Customer transformer to Select features 
class ColumnExtractor(TransformerMixin):
    def __init__(self, cols):
        self.cols = cols

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xcols = pd.Series(X[self.cols])
        return (Xcols)


In [79]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator


nlp = en_core_web_sm.load()


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 
                    variety="BrE",
                 user_abbrevs={},
                 n_jobs=1):
        """
        Text preprocessing transformer includes steps:
            1. Text normalization
            2. Punctuation removal
            3. Stop words removal
            4. Lemmatization
        
        variety - format of date (AmE - american type, BrE - british format) 
        user_abbrevs - dict of user abbreviations mappings (from normalise package)
        n_jobs - parallel jobs to run
        """
        self.variety = variety
        self.user_abbrevs = user_abbrevs
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()

        partitions = 1
        cores = mp.cpu_count()
        if self.n_jobs <= -1:
            partitions = cores
        elif self.n_jobs <= 0:
            return X_copy.apply(self._preprocess_text)
        else:
            partitions = min(self.n_jobs, cores)

        data_split = np.array_split(X_copy, partitions)
        pool = mp.Pool(cores)
        data = pd.concat(pool.map(self._preprocess_part, data_split))
        pool.close()
        pool.join()

        return data

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        normalized_text = self._normalize(text)
        doc = nlp(normalized_text)

        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _normalize(self, text):
        # some issues in normalise package
        try:
            return ' '.join(normalise(text, variety=self.variety, user_abbrevs=self.user_abbrevs, verbose=False))
        except:
            return text

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])



num_cols = ["punct_pc","text_length"]
Column_trans = ColumnTransformer(
     [('scaler', StandardScaler(),num_cols)],
     remainder='drop')

num_pipe = Pipeline([
  ('scaler', Column_trans)
  ])

In [80]:
X_train.head()

,text,punct_pc,text_length
1627,You have been selected to stay in 1 of 250 top British hotels - FOR NOTHING! Holiday Worth å£350...,4.958678,121
4608,Y de asking like this.,5.555556,18
2709,Sorry de i went to shop.,5.263158,19
639,I had askd u a question some hours before. Its answer,2.325581,43
4544,Never y lei... I v lazy... Got wat? Dat day Ì_ send me da url cant work one...,18.032787,61


In [81]:
Preprocess_text = Pipeline([("select_text", ColumnExtractor(cols="text")),
                            ('preprocess', TextPreprocessor())
                            ])

preprocessing_pipeline = Pipeline([
    ('feat_union', FeatureUnion(transformer_list=[
          ('text_pipeline', Preprocess_text),
          ('num_pipeline', num_pipe)
          ]))
    ])

**Preprocess the Train and Test data using the pipeline**

In [82]:
X_train_preprocessed = Preprocess_text.fit_transform(X_train)
X_test_preprocessed = Preprocess_text.fit_transform(X_test)

Calculate the size of the vocabulary (i.e the number of unique words in the entire corpus) . This is needed as a paramter "input_dim" in the embedded layer 

In [83]:
from collections import Counter
# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

counter = counter_word(X_train_preprocessed)
vocab_size = len(counter)

In [84]:
# We are taking a maximum lenght for the sequence as 30 (sequence above this length will be trimmed down and 
# and below this lenght will be padded with zeroes)
max_length = 30
num_words = vocab_size + 1000

Using Tokenizer class from keras tokenize the sentences. Tokinzer assignes a unique ID for each word in the entire trainign set

In [85]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train_preprocessed)
X_train_sequences = tokenizer.texts_to_sequences(X_train_preprocessed)

In [86]:
## We can check the index of all the words created by the tokenizer
word_index = tokenizer.word_index
#print(word_index)

Using pad_squeence, pad the sequences to have the same length

In [87]:
from keras.preprocessing.sequence import pad_sequences
X_train_padded = pad_sequences(
    X_train_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [88]:
## Applying same tranformation to Test set
test_sequences = tokenizer.texts_to_sequences(X_test_preprocessed)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

## Deep Learning model training

In [89]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

model = Sequential()

model.add(Embedding(num_words, 32, input_length=max_length))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))


optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [90]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 32)            297408    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 322,305
Trainable params: 322,305
Non-trainable params: 0
_________________________________________________________________


In [91]:
Trained_model = model.fit(
    X_train_padded, y_train, epochs=10, validation_data=(test_padded, y_test)
)

Epoch 1/10
140/140 [==============================] - 6s 29ms/step - loss: 0.5252 - accuracy: 0.7927 - val_loss: 0.1318 - val_accuracy: 0.9561
Epoch 2/10
140/140 [==============================] - 3s 25ms/step - loss: 0.0875 - accuracy: 0.9763 - val_loss: 0.0526 - val_accuracy: 0.9883
Epoch 3/10
140/140 [==============================] - 3s 25ms/step - loss: 0.0274 - accuracy: 0.9961 - val_loss: 0.0459 - val_accuracy: 0.9874
Epoch 4/10
140/140 [==============================] - 4s 26ms/step - loss: 0.0160 - accuracy: 0.9972 - val_loss: 0.0501 - val_accuracy: 0.9883
Epoch 5/10
140/140 [==============================] - 4s 26ms/step - loss: 0.0083 - accuracy: 0.9984 - val_loss: 0.0436 - val_accuracy: 0.9883
Epoch 6/10
140/140 [==============================] - 4s 26ms/step - loss: 0.0081 - accuracy: 0.9985 - val_loss: 0.0520 - val_accuracy: 0.9883
Epoch 7/10
140/140 [==============================] - 4s 26ms/step - loss: 0.0029 - accuracy: 0.9996 - val_loss: 0.0507 - val_accuracy: 0.9883

In [92]:
model.save("trained_model.h5")

# **Predict on New Test Data**

In [106]:
test_data = pd.read_csv("Test_Emails.csv",names=['label','text'], encoding='ISO-8859-1')

In [110]:
test_data

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
...,...,...
490,ham,"Sorry man my account's dry or I would, if you want we could trade back half or I could buy some ..."
491,spam,"Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3,..."
492,ham,"Sorry,in meeting I'll call later"
493,ham,What class of &lt;#&gt; reunion?


In [115]:
new_X_test = test_data.loc[:,["text"]]
new_Y_test = test_data.loc[:,["label"]]

In [116]:
new_Y_test

,label
0,ham
1,ham
2,spam
3,ham
4,ham
...,...
490,ham
491,spam
492,ham
493,ham


In [117]:
new_X_test.shape

(495, 1)

In [120]:
Y_test = np.where(new_Y_test.label == 'spam', 1, 0)

**Preprocess Test data by fitting in the preprocess pipeline**

In [102]:
## Fit X_test in pipeline
new_X_test_processes = Preprocess_text.fit_transform(new_X_test)

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(new_X_test_processes)
new_X_test_sequences = tokenizer.texts_to_sequences(new_X_test_processes)


from keras.preprocessing.sequence import pad_sequences
new_X_test_padded = pad_sequences(
    new_X_test_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [103]:
predict = model.predict_classes(new_X_test_padded)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [122]:
## Performance metrics
from sklearn.metrics import classification_report
print(classification_report(Y_test,predict))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       424
           1       0.15      0.11      0.13        71

    accuracy                           0.78       495
   macro avg       0.51      0.50      0.50       495
weighted avg       0.76      0.78      0.77       495



In [123]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test,predict))

[[380  44]
 [ 63   8]]
